In [ ]:
from asbe.base import *
from asbe.models import *
from asbe.estimators import *
from econml.orf import DMLOrthoForest
from econml.dml import CausalForestDML
#from causalml.inference.nn import CEVAE
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from copy import deepcopy
import econml
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from sklift.datasets import fetch_megafon

In [ ]:
# np.random.seed(1005)
# y, X, t, ite, _, e = synthetic_data(mode=1, n=1000, p=5, sigma=1.0)

In [ ]:
X, y, t = fetch_megafon(return_X_y_t=True)

In [ ]:
X = X.to_numpy()[:1000,:]
y = y.to_numpy()[:1000]
t = t.to_numpy()[:1000]
t = np.where(t == "treatment", 1, 0)

In [ ]:
# y1 = np.where(t == 1, y, y + ite)
# y0 = np.where(t == 0, y, y - ite)
X_train, X_test, t_train, t_test, y_train, y_test = train_test_split(
    X, t, y,  test_size=0.9, random_state=1005)
ds = {"X_training": X_train,
     "y_training": y_train,
     "t_training": t_train,
     "X_pool": deepcopy(X_test), 
     "y_pool": deepcopy(y_test),
     "t_pool": deepcopy(t_test),
     "y1_pool": y1_test,
     "y0_pool":y0_test,
     "X_test": X_test,
     "y_test": y_test,
      "t_test": t_test,
      "ite_test": ite_test
     }

In [ ]:
def test_acq(estimator, acq):
    asl = BaseActiveLearner(estimator = BARTstimator(model = estimator,
                                         two_model=False,ps_model=None),
                        acquisition_function=acq,
                        assignment_function=UncertaintyAssignmentFunction(),
                        stopping_function = None,
                        dataset=ds)
    asl.fit()
    X_new, query_idx = asl.query(no_query=10)
    print(query_idx)
    asl.teach(query_idx)
    preds = asl.predict(asl.dataset["X_test"])
    print(asl.score(metric="Qini"))
    return True

# def test_acq_ob(acq):
#     asl = BaseActiveLearner(estimator = OPENBTITEEstimator(model = OPENBT(
#         model="bart",ntrees=200),
#                                          two_model=False,ps_model=None),
#                         acquisition_function=acq,
#                         assignment_function=MajorityAssignmentFunction(),
#                         stopping_function = None,
#                         dataset=ds)
#     asl.fit()
#     X_new, query_idx = asl.query(no_query=10)
#     print(query_idx)
#     asl.teach(query_idx)
#     preds = asl.predict(asl.dataset["X_test"])
#     print(asl.score())
#     return True

def test_acq_gp(acq):
    asl = BaseActiveLearner(estimator = GPEstimator(model = GaussianProcessRegressor(ConstantKernel()*RBF(np.ones(ds["X_training"].shape[1],))),
                                         two_model=True,
                                                    ps_model=None),
                        acquisition_function=acq,
                        assignment_function=MajorityAssignmentFunction(),
                        stopping_function = None,
                        dataset=ds)
    asl.fit()
    X_new, query_idx = asl.query(no_query=20)
    print(query_idx)
    print(asl.score())
    asl.teach(query_idx)
    asl.fit()
    #preds = asl.predict(asl.dataset["X_test"])
    print(asl.score())
    return True

In [ ]:
test_acq(OPENBT(model="bart"), RandomAcquisitionFunction())

Overwriting k to agree with the model's default
Overwriting overallnu to agree with the model's default
Overwriting ntree to agree with the model's default
Overwriting ntreeh to agree with the model's default
Overwriting overallsd to agree with the model's default
Writing config file and data
/var/folders/44/gtm_t6x110jg6b13p4rbwkfh0000gn/T/openbtpy_ftkizzo6
3+ x variables
Running model...
[699 609 381 649 806 842 805 314 112 283]
0.04906324638964392


True

In [ ]:
# test_acq(RandomForestRegressor(), UncertaintyAcquisitionFunction())

In [ ]:
#test_acq_ob(UncertaintyAcquisitionFunction())

In [ ]:
#test_acq_ob(TypeSAcquistionFunction())

In [ ]:
#test_acq_ob(EMCMAcquisitionFunction(no_query=10, B=10))

In [ ]:
#test_acq_gp(RandomAcquisitionFunction())

In [ ]:
test_acq_gp(UncertaintyAcquisitionFunction())

[687 421 116 185 205 482 486 440 635 243  76 476 869 612 506 108 530 611
 268 368]
0.5337187351959323
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 0.53038938 -0.55797198  1.52225302 -0.29576746 -0.21913841  0.79679204
  1.3020338   3.89374152  1.08700095  1.1477019   2.21613025  1.09288272
  2.93490226  1.64948686  2.07372609  0.98864345  1.93008768  0.86036131
 -0.04415296  0.48432562]
[ 1.21845907 -0.32675738  1.92124318  0.43299755 -0.00447676  1.56619227
  1.84402239  4.44810804  1.59475614  1.40139785  2.56874432  1.85665729
  3.75399006  2.41448468  2.9024177   1.75861399  2.86750032  1.65045353
  0.22963051  0.74852614]
[ 0.53038938 -0.55797198  1.52225302 -0.29576746 -0.21913841  0.79679204
  1.3020338   3.89374152  1.08700095  1.1477019   2.21613025  1.09288272
  2.93490226  1.64948686  2.07372609  0.98864345  1.93008768  0.86036131
 -0.04415296  0.48432562]
0.5751091655772831


True

In [ ]:
test_acq_gp(TypeSAcquistionFunction())

[300 756 111  71 769 390 612 371 122 360 135 352 835 337 624 141 636 316
 529 142]
0.5337187351959323
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-1.36067322e-03 -5.12796861e-01  1.02289924e+00  8.38177760e-01
  1.44924400e+00  2.57422847e+00  1.64948686e+00  7.30995761e-01
  2.59964492e+00  7.20538605e-01  1.66191105e+00  2.78676717e-01
  5.00527092e-01  1.79809835e+00  3.03647193e+00  2.71252851e+00
  2.58165302e+00  2.38965847e+00  2.94782426e+00  1.25635611e+00]
[ 0.33369527 -0.33240087  1.21676014  1.56122575  1.75384184  3.40890769
  2.41448468  1.21785526  2.83841668  1.2186425   2.36359886  1.08998568
  1.05927929  2.5266249   3.41148707  3.386568    3.20981977  3.26452459
  3.29757026  1.64321664]
[-1.36067322e-03 -5.12796861e-01  1.02289924e+00  8.38177760e-01
  1.44924400e+00  2.57422847e+00  1.64948686e+00  7.30995761e-01
  2.59964492e+00  7.20538605e-01  1.66191105e+00  2.78676717e-01
  5.00527092e-01  1.79809835e+00  3.03647193e+00  2.71252851e+00
  2.5

True

In [ ]:
test_acq_gp(EMCMAcquisitionFunction())

[199 899 308 306 305 304 303 302 301 300 299 298 297 296 295 294 293 292
 291 290]
0.5337187351959323
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 1.24313648e-01  1.52989324e+00  1.96915558e-01  1.02021858e+00
  1.52304711e+00  9.65934285e-01  7.98132313e-01  1.52358418e+00
  1.40168734e+00 -1.36067322e-03  1.94210522e+00  7.15502131e-01
 -1.32016499e+00  3.50806262e-02  2.86028182e+00  2.80033832e+00
 -5.83705994e-02 -8.77760421e-01 -4.94267359e-01  2.33824913e+00]
[ 6.61634423e-01  1.92266739e+00  8.08301031e-01  1.63039998e+00
  1.97606387e+00  1.57041756e+00  1.44214321e+00  1.63268067e+00
  1.57287874e+00  3.33695275e-01  2.28576981e+00  1.11985145e+00
 -8.27985526e-01  7.35931053e-01  3.64530045e+00  3.32170215e+00
  1.93474920e-01  3.13327052e-03 -3.62252875e-01  3.18616992e+00]
[ 1.24313648e-01  1.52989324e+00  1.96915558e-01  1.02021858e+00
  1.52304711e+00  9.65934285e-01  7.98132313e-01  1.52358418e+00
  1.40168734e+00 -1.36067322e-03  1.94210522e+00  7.15

True